In [6]:
from ortools.sat.python import cp_model
import numpy as np
import time

In [7]:
def convert_string_int(s):
    rs = s.split(" ")
    rs = [int(x) for x in rs]
    return rs


In [8]:
class Solver():
    def __init__(self,file_path): 
        with open(file=file_path) as f:
            line1 = f.readline()
            self.N, self.D = convert_string_int(line1)
            self.w = np.ones((self.N+1,self.N+1,self.D+1))*99
            line2 = f.readline()
            self.s, self.t = convert_string_int(line2)
            lines = f.readlines()
            for line in lines:
                s,t,w,c = convert_string_int(line)
                self.w[s][t][c] = min(self.w[s][t][c],w)
        f.close()
        self.model = cp_model.CpModel()
    def solve(self):
        timestart = time.time()
        x= {}
        for i in range(1, self.N+1):
            for j in range(1, self.N+1):
                for k in range(1,self.D+1):
                    x[str(i) + "," + str(j) + "," + str(k)] = self.model.NewIntVar(0, 1, "x[" + str(i) + "," + str(j)+ "," + str(k) + "]")

        t = {}
        for i in range(1,self.N+1):
            for j in range(1,self.D+1):
                t[str(i) + "," + str(j)] = self.model.NewIntVar(0, 1, "t[" + str(i) + "," + str(j) + "]")

        p = {}

        for i in range(1,self.D+1):
            p[str(i)] = self.model.NewIntVar(0,self.D, "p[" + str(i) + "]")
        for i in range(1,self.N+1):
            in_deg = x[str(1) + "," + str(i) + "," + str(1)]*1
            out_deg = x[str(i) + "," + str(1) + "," + str(1)]*1
            for j in range(1,self.N+1):
                for k in range(1,self.D+1):
                    if j == 1 and k == 1: 
                        continue
                    in_deg += x[str(j) + "," + str(i)+ "," + str(k)]
                    out_deg += x[str(i) + "," + str(j)+"," + str(k)]
            if i == self.s: 
                self.model.Add(in_deg == 0)
                self.model.Add(out_deg == 1)
            elif i == self.t:
                self.model.Add(in_deg == 1)
                self.model.Add(out_deg == 0)
            else:
                self.model.Add(in_deg == out_deg)
                self.model.Add(in_deg <=1)
                self.model.Add(out_deg <= 1)
            for k in range(1,self.D+1):
                exp = x[str(1) + ","+str(1)+","+str(k)]*1
                for i in range(1,self.N+1):
                    for j in range(1,self.N+1):  
                        exp += x[str(1) + ","+str(1)+","+str(k)]
                self.model.Add(p[str(k)] == exp)

            for i in range(1,self.N+1):
                for k in range(1,self.D+1):
                    exp = x[str(i) + ","+str(1)+","+str(k)]*1
                    exp+=x[str(1) + ","+str(i)+","+str(k)]*1
                    a = self.model.NewIntVar(0,1,"a")
                    self.model.AddMultiplicationEquality(a,[x[str(i) + ","+str(1)+","+str(k)],x[str(1) + ","+str(i)+","+str(k)]])
                    exp-=a*1
                    for j in range(2,self.N+1):
                        exp += x[str(i) + ","+str(j)+","+str(k)]
                        exp += x[str(j) + ","+str(i)+","+str(k)]
                        b = self.model.NewIntVar(0,1,"b")
                        self.model.AddMultiplicationEquality(a,[x[str(i) + ","+str(j)+","+str(k)],x[str(j) + ","+str(i)+","+str(k)]])
                        exp-= b*1
                    self.model.Add(exp == t[str(i)+","+str(k)])
            for k in range(1,self.D+1):
                exp = t[str(1)+","+str(k)]*1
                for i in range(2,self.N+1):
                    exp+= t[str(i)+","+str(k)]
                self.model.Add(exp - p[str(k)] <= 1)
                for k in range(1,self.D+1):
                    exp = t[str(1)+","+str(k)]*1
                    for i in range(2,self.N+1):
                        exp+= t[str(i)+","+str(k)]
                    self.model.Add(exp - p[str(k)] <= 1)
            func = self.model.NewIntVar(0,1000,"func")
            for i in range(1,self.N+1):
                for j in range(1,self.N+1):
                    for k in range(1,self.D+1):
                        func+= x[str(i) + ","+str(j)+","+str(k)] * self.w[i][j][k]
        self.model.Minimize(func)
        solver = cp_model.CpSolver()
        status = solver.Solve(self.model)
        if status == cp_model.OPTIMAL:
            print(solver.ObjectiveValue())
        print('Statistics')
        print(f'  status   : {solver.StatusName(status)}')
        print(f'  conflicts: {solver.NumConflicts()}')
        print(f'  branches : {solver.NumBranches()}')
        print(f'  wall time: {solver.WallTime()} s')
        print(f'  Run time: {time.time()-timestart} s')

In [9]:
fpath =  'MFEA_lib/tasks/Benchmark/__references__/IDPC_DU/IDPC_EDU/data/set1/idpc_10x5x425.idpc'
sol = Solver(file_path =fpath)
sol.solve()

7.0
Statistics
  status   : OPTIMAL
  conflicts: 0
  branches : 802
  wall time: 0.2044582 s
  Run time: 1.690011739730835 s


In [10]:
import os
path = "MFEA_lib/tasks/Benchmark/__references__/IDPC_DU/IDPC_EDU/data/set1"
dir_list = os.listdir(path)[:8]
for dir in dir_list:
    fpath = path+ '/' + dir
    print('\n'+dir)
    sol = Solver(fpath)
    sol.solve()



idpc_10x10x1000.idpc
7.0
Statistics
  status   : OPTIMAL
  conflicts: 1
  branches : 1687
  wall time: 0.42651520000000004 s
  Run time: 6.2127697467803955 s

idpc_10x20x2713.idpc
7.0
Statistics
  status   : OPTIMAL
  conflicts: 1
  branches : 4310
  wall time: 1.015035316 s
  Run time: 7.012154817581177 s

idpc_10x5x425.idpc
7.0
Statistics
  status   : OPTIMAL
  conflicts: 0
  branches : 802
  wall time: 0.19434399300000002 s
  Run time: 1.4717695713043213 s

idpc_15x15x3375.idpc
10.0
Statistics
  status   : OPTIMAL
  conflicts: 50
  branches : 7083
  wall time: 2.699073941 s
  Run time: 26.075262546539307 s

idpc_15x30x12111.idpc
10.0
Statistics
  status   : OPTIMAL
  conflicts: 12
  branches : 12706
  wall time: 7.032297793000001 s
  Run time: 49.986276149749756 s

idpc_15x7x1504.idpc
10.0
Statistics
  status   : OPTIMAL
  conflicts: 9
  branches : 3209
  wall time: 1.085415441 s
  Run time: 8.365888118743896 s

idpc_20x10x2492.idpc
15.0
Statistics
  status   : OPTIMAL
  conflicts:

## Biến
- X(i,j,k) = 1, nếu chu trình đi từ đỉnh i đến đỉnh j theo cạnh màu k

- In_deg(i) = $\sum_{j=1}^{N}$ $\sum_{k=1}^{D}  X(j,i,k)$  : số cạnh đi vào đỉnh i 

- Out_deg(i) = $\sum_{j=1}^{N}$ $\sum_{k=1}^{D}  X(i,j,k)$ : số cạnh đi ra khỏi đỉnh i 

- t(i,k)  = 1 nếu có đường đi ra/ hoặc vào đỉnh i có màu k 

- p(k) : số đường có màu k trong chu trình
## Ràng buộc
1. Mỗi đỉnh chỉ đi qua tối đa 1 lần và bắt buộc phải xuất phát tại s và kết thúc t:
    - In_deg(s) = Out_deg(t) = 0
    
    - Out_deg(s) = In_deg(t) = 1

    - In_deg(i) = Out_deg(i) $\forall i \in [1,N]$ \ {s,t}

    - In_deg(i),Out_deg(i) $\in [0,1]$ $\forall i \in [1,N]$

2. Mỗi miền chỉ đi qua tối đa 1 lần:

    - p(k) = $\sum_{i=1}^{N}$ $\sum_{j=1}^{N}  X(j,i,k)$

    - t(i,k) = $\sum_{j=1}^{N} X(i,j,k)+X(j,i,k) - X(i,j,k)*X(j,i,k)$ 

    - $\sum_{i=1}^{N} \Big(t(i,k)\Big) - p(k) \le 1 $

$\sum_{j=1}^{N}$ $\sum_{k=1}^{D}  x[j][i][k]$